# DATASET INTEGRATION

In [1]:
# Import required libraries.
import pandas as pd

df = pd.read_csv('../datasets/customer_shopping_data_v0.csv')

#### v1 = date version is transformed from DD-MM-YY to YYYY-MM-DD

In [9]:
# Save the processed data.
df['invoice_date'] = pd.to_datetime(df['invoice_date'], dayfirst=True)
df.to_csv('../datasets/customer_shopping_data_v1.csv', index=False)

#### v2 = 'season' column is added

In [10]:
# Load dataset from file.
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Null'

df = pd.read_csv('../datasets/customer_shopping_data_v1.csv')
df['season'] = pd.to_datetime(df['invoice_date']).dt.month.apply(get_season)

df.to_csv('../datasets/customer_shopping_data_v2.csv', index=False)
df = pd.read_csv('../datasets/customer_shopping_data_v2.csv')

#### v3 = 'is_weekday' column is added

In [13]:
# Create or transform features.
def is_weekday(day):
    if day < 5:  # 0-4 -> Monday - Friday
        return 1
    else:        # 5–6 -> Saturday - Sunday
        return 0

df['is_weekday'] = pd.to_datetime(df['invoice_date']).dt.weekday.apply(is_weekday)

df.to_csv('../datasets/customer_shopping_data_v3.csv', index=False)

#### v4 = 'is_holiday' column is added

In [1]:
# Import required libraries.
import pandas as pd
import holidays

df = pd.read_csv('../datasets/customer_shopping_data_v3.csv')
df['invoice_date'] = pd.to_datetime(df['invoice_date'])

date_range = pd.date_range(start='2021-01-01', end='2023-12-31')
df_dates = pd.DataFrame({'date': date_range})

turkiye_holidays = holidays.Turkey(years=[2021, 2022, 2023])
df_dates['is_official_holiday'] = df_dates['date'].dt.date.isin(set(turkiye_holidays.keys()))
df_dates['is_weekend'] = df_dates['date'].dt.weekday >= 5

manual_full_holidays = [
]
manual_full_holidays = pd.to_datetime(manual_full_holidays).date
df_dates['is_manual_full'] = df_dates['date'].dt.date.isin(manual_full_holidays)

manual_half_holidays = [
]
manual_half_holidays = pd.to_datetime(manual_half_holidays).date
df_dates['is_manual_half'] = df_dates['date'].dt.date.isin(manual_half_holidays)

df_dates['is_holiday'] = 0
df_dates.loc[df_dates['is_manual_half'], 'is_holiday'] = 2
df_dates.loc[df_dates['is_official_holiday'] | df_dates['is_weekend'] | df_dates['is_manual_full'], 'is_holiday'] = 1

df_dates = df_dates[['date', 'is_holiday']]
df = df.merge(df_dates, left_on='invoice_date', right_on='date', how='left')
df.drop(columns=['date'], inplace=True)

df.to_csv('../datasets/customer_shopping_data_v4.csv', index=False)

#### v5 = economic indicators' columns are added by their date equality

In [8]:
# Import required libraries.
import pandas as pd

df_shopping = pd.read_csv("../datasets/customer_shopping_data_v4.csv")
df_index = pd.read_csv("../datasets/economic_confident_indexes_extracted.csv", sep=";")

for col in ["Econ_Conf", "Cons_Conf", "RealSec_Conf", "Serv_Conf", "Retail_Conf", "Constr_Conf"]:
    df_index[col] = df_index[col].str.replace(",", ".").astype(float)

df_shopping["invoice_date"] = pd.to_datetime(df_shopping["invoice_date"])
df_shopping["Year"] = df_shopping["invoice_date"].dt.year
df_shopping["Month"] = df_shopping["invoice_date"].dt.month

df_merged = pd.merge(df_shopping, df_index, how="left", on=["Year", "Month"])
df_merged.drop(["Year", "Month"], axis=1, inplace=True)

df_merged.to_csv("../datasets/customer_shopping_data_v5.csv", index=False)

#### v6 = 'total_price' column added

In [2]:
df = pd.read_csv("../datasets/customer_shopping_data_v5.csv") 

df["total_price"] = df["price"] * df["quantity"]

df.to_csv("../datasets/customer_shopping_data_v6.csv", index=False)

In [3]:
import pandas as pd

# Drop list
drop_cols = ['Serv_Conf', 'Retail_Conf', 'Constr_Conf']

# File mapping → hangi v dosyası → hangi clean versiyona kaydedilecek
files = {
    'v6': {
        'path': '../datasets/customer_shopping_data_v6.csv',
        'output': '../datasets/customer_shopping_data_v10.csv'
    },
    'v7': {
        'path': '../datasets/customer_shopping_data_v7_weekly.csv',
        'output': '../datasets/customer_shopping_data_v11_weekly.csv'
    },
    'v8': {
        'path': '../datasets/customer_shopping_data_v8_monthly.csv',
        'output': '../datasets/customer_shopping_data_v12_monthly.csv'
    },
    'v9': {
        'path': '../datasets/customer_shopping_data_v9_yearly.csv',
        'output': '../datasets/customer_shopping_data_v13_yearly.csv'
    }
}

# Process each file
for version, info in files.items():
    df = pd.read_csv(info['path'])
    df_clean = df.drop(columns=[col for col in drop_cols if col in df.columns])
    df_clean.to_csv(info['output'], index=False)
    print(f"[{version}] Saved clean version: {info['output']}")


[v6] Saved clean version: ../datasets/customer_shopping_data_v10.csv
[v7] Saved clean version: ../datasets/customer_shopping_data_v11_weekly.csv
[v8] Saved clean version: ../datasets/customer_shopping_data_v12_monthly.csv
[v9] Saved clean version: ../datasets/customer_shopping_data_v13_yearly.csv


In [21]:
import pandas as pd

# Load indices
bloomberg = pd.read_csv("../datasets/bloomberg_cons_conf_index.csv", sep=';')
oecd = pd.read_csv("../datasets/oecd_cons_conf_index.csv")
ipsos = pd.read_csv("../datasets/ipsos_cons_conf_index.csv")

# Prepare Ipsos
ipsos["Date"] = pd.to_datetime(ipsos["Date"])
ipsos["year"] = ipsos["Date"].dt.year
ipsos["month"] = ipsos["Date"].dt.month
if pd.api.types.is_numeric_dtype(ipsos["PCSI"]):
    ipsos["ipsos_confidence"] = ipsos["PCSI"]
else:
    ipsos["ipsos_confidence"] = ipsos["PCSI"].str.replace(",", ".").astype(float)

# Merge indices into a single dataframe
indices = (
    pd.merge(bloomberg, oecd, on=["year", "month"], how="outer")
    .merge(ipsos[["year", "month", "ipsos_confidence"]], on=["year", "month"], how="outer")
)

# Compute yearly averages
yearly_avg_all = (
    indices
    .groupby("year")
    .agg({
        "bloomberg_confidence": "mean",
        "oecd_confidence": "mean",
        "ipsos_confidence": "mean"
    })
    .reset_index()
)

# For 2023: Jan-Mar only
yearly_avg_2023 = (
    indices.loc[(indices["year"] == 2023) & (indices["month"] <=3)]
    .agg({
        "bloomberg_confidence": "mean",
        "oecd_confidence": "mean",
        "ipsos_confidence": "mean"
    })
    .to_frame()
    .T
)
yearly_avg_2023["year"] = 2023

yearly_avg = pd.concat([
    yearly_avg_all[yearly_avg_all["year"] != 2023],
    yearly_avg_2023
], ignore_index=True)

# Merge helpers
def merge_monthly(df, date_col):
    df[date_col] = pd.to_datetime(df[date_col])
    df["year"] = df[date_col].dt.year
    df["month"] = df[date_col].dt.month
    merged = df.merge(
        indices,
        on=["year", "month"],
        how="left"
    )
    return merged.drop(columns=["year", "month"])

def merge_yearly(df, date_col):
    df[date_col] = pd.to_datetime(df[date_col])
    df["year"] = df[date_col].dt.year
    merged = df.merge(
        yearly_avg,
        on="year",
        how="left"
    )
    return merged.drop(columns=["year"])

# DAILY
df_v10 = pd.read_csv("../datasets/customer_shopping_data_v10.csv")
df_v10 = merge_monthly(df_v10, "invoice_date")
df_v10.to_csv("../datasets/customer_shopping_data_v14.csv", index=False)

# WEEKLY
df_v11 = pd.read_csv("../datasets/customer_shopping_data_v11_weekly.csv")
df_v11 = merge_monthly(df_v11, "week_key")
df_v11.to_csv("../datasets/customer_shopping_data_v15_weekly.csv", index=False)

# MONTHLY
df_v12 = pd.read_csv("../datasets/customer_shopping_data_v12_monthly.csv")
df_v12 = merge_monthly(df_v12, "month_key")
df_v12.to_csv("../datasets/customer_shopping_data_v16_monthly.csv", index=False)

# YEARLY
df_v13 = pd.read_csv("../datasets/customer_shopping_data_v13_yearly.csv")
df_v13 = merge_yearly(df_v13, "year_key")
df_v13.to_csv("../datasets/customer_shopping_data_v17_yearly.csv", index=False)

print("✅ All datasets processed successfully.")


✅ All datasets processed successfully.
